Conjuntos de entrenamiento y prueba<br>
Ya hemos visto cómo ajustar un modelo a un conjunto de datos. En este ejercicio, veremos cómo comprobar y confirmar la validez y el rendimiento de nuestros modelos utilizando conjuntos de entrenamiento y de prueba. Como de costumbre, empezamos por cargar y echar un vistazo a nuestros datos:

In [ ]:
import pandas
!pip install statsmodels
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/graphing.py
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/dog-training.csv
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/dog-training-switzerland.csv

data = pandas.read_csv("dog-training.csv", delimiter="\t")

print(data.shape)
print(data.head())

Nos interesa la relación entre el peso de un perro y la cantidad de rescates que realizó el año anterior. Empecemos trazando rescates_último_año en función de peso_último_año:

In [ ]:
import graphing
import statsmodels.formula.api as smf

# First, we define our formula using a special syntax
# This says that rescues_last_year is explained by weight_last_year
formula = "rescues_last_year ~ weight_last_year"

model = smf.ols(formula = formula, data = data).fit()

graphing.scatter_2D(data, "weight_last_year", "rescues_last_year", trendline = lambda x: model.params[1] * x + model.params[0])

Parece haber una relación bastante clara entre el peso de un perro y el número de rescates que ha realizado. Esto parece bastante razonable, ya que cabría esperar que los perros más pesados fueran más grandes y fuertes y, por tanto, mejores salvando vidas.

Partición de entrenamiento/prueba<br>
Esta vez, en lugar de ajustar un modelo a la totalidad de nuestro conjunto de datos, vamos a separar nuestro conjunto de datos en dos particiones más pequeñas: un conjunto de entrenamiento y un conjunto de prueba.
El conjunto de entrenamiento es el más grande de los dos, y suele estar formado por entre el 70 y el 80% del conjunto de datos, mientras que el resto constituye el conjunto de prueba. Al dividir los datos, podemos medir el rendimiento de nuestro modelo cuando se enfrenta a datos que no se han visto antes.

Tenga en cuenta que los datos del conjunto de prueba nunca se utilizan en el entrenamiento. Por eso se suelen denominar datos no vistos o datos desconocidos por el modelo.

In [ ]:
from sklearn.model_selection import train_test_split


# Obtain the label and feature from the original data
dataset = data[['rescues_last_year','weight_last_year']]

# Split the dataset in an 70/30 train/test ratio. We also obtain the respective corresponding indices from the original dataset.
train, test = train_test_split(dataset, train_size=0.7, random_state=21)

print("Train")
print(train.head())
print(train.shape)

print("Test")
print(test.head())

Podemos ver que estos conjuntos son diferentes, y que el conjunto de entrenamiento y el conjunto de prueba contienen el 70% y el 30% de los datos totales, respectivamente.

Veamos cómo se separan el conjunto de entrenamiento y el conjunto de prueba:

In [ ]:
# You don't need to understand this code well
# It's just used to create a scatter plot

# concatenate training and test so they can be graphed
plot_set = pandas.concat([train,test])
plot_set["Dataset"] = ["train"] * len(train) + ["test"] * len(test)

# Create graph
graphing.scatter_2D(plot_set, "weight_last_year", "rescues_last_year", "Dataset", trendline = lambda x: model.params[1] * x + model.params[0])

Conjunto de entrenamiento<br>
Comenzamos entrenando nuestro modelo utilizando el conjunto de entrenamiento y probamos su rendimiento con el mismo conjunto de entrenamiento:

In [ ]:
import statsmodels.formula.api as smf
from sklearn.metrics import mean_squared_error as mse

# First, we define our formula using a special syntax
# This says that rescues_last_year is explained by weight_last_year
formula = "rescues_last_year ~ weight_last_year"

# Create and train the model
model = smf.ols(formula = formula, data = train).fit()

# Graph the result against the data
graphing.scatter_2D(train, "weight_last_year", "rescues_last_year", trendline = lambda x: model.params[1] * x + model.params[0])

Podemos medir el rendimiento de nuestro modelo calculando el error cuadrático medio (ECM)

In [ ]:
# We use the in-buit sklearn function to calculate the MSE
correct_labels = train['rescues_last_year']
predicted = model.predict(train['weight_last_year'])

MSE = mse(correct_labels, predicted)
print('MSE = %f ' % MSE)

Conjunto de pruebas
A continuación, probamos el rendimiento del mismo modelo utilizando el conjunto de pruebas:

In [ ]:
graphing.scatter_2D(test, "weight_last_year", "rescues_last_year", trendline = lambda x: model.params[1] * x + model.params[0])


Echemos un vistazo de nuevo al MSE.

In [ ]:
correct_labels = test['rescues_last_year']
predicted = model.predict(test['weight_last_year'])

MSE = mse(correct_labels, predicted)
print('MSE = %f ' % MSE)

Podemos ver que el modelo funciona mucho mejor con los datos de entrenamiento conocidos que con los datos de prueba desconocidos (recuerde que los valores de MSE más altos son peores).

Esto puede deberse a varios factores, pero el primero y más importante es el sobreajuste, que se produce cuando un modelo se ajusta demasiado a los datos del conjunto de entrenamiento. Esto significa que funcionará muy bien en el conjunto de entrenamiento, pero no generalizará bien. (es decir, no funcionará bien con otros conjuntos de datos).

Nuevo conjunto de datos<br>
Para ilustrar mejor nuestra idea, veamos cómo se comporta nuestro modelo ante un conjunto de datos completamente nuevo, desconocido y de mayor tamaño. Para nuestro escenario, utilizaremos los datos proporcionados por la rama europea de la organización benéfica de rescate en avalanchas.

In [ ]:
# Load an alternative dataset from the charity's European branch
new_data = pandas.read_csv("dog-training-switzerland.csv", delimiter="\t")

print(new_data.shape)
new_data.head()


Las características son las mismas, pero esta vez tenemos muchos más datos. Veamos cómo funciona nuestro modelo.

In [ ]:
# Plot the fitted model against this new dataset. 

graphing.scatter_2D(new_data, "weight_last_year", "rescues_last_year", trendline = lambda x: model.params[1] * x + model.params[0])

Y ahora el MSE:

In [ ]:
correct_labels = new_data['rescues_last_year']
predicted = model.predict(new_data['weight_last_year'])

MSE = mse(correct_labels, predicted)
print('MSE = %f ' % MSE)

Como era de esperar, el modelo funciona mejor en el conjunto de datos de entrenamiento que en el conjunto de datos no visto. Esto se debe simplemente al sobreajuste, como vimos anteriormente.

Curiosamente, el modelo funciona mejor en este conjunto de datos que en el de prueba. Esto se debe a que nuestro conjunto de pruebas anterior era bastante pequeño y, por tanto, no representaba muy bien los datos del "mundo real". En cambio, este conjunto de datos es grande y representa mucho mejor los datos que encontraremos fuera del laboratorio. En esencia, esto nos muestra que parte de la diferencia de rendimiento que vemos entre el entrenamiento y la prueba se debe al sobreajuste del modelo, y parte del error se debe a que el conjunto de prueba no es perfecto. En los próximos ejercicios, exploraremos el equilibrio que tenemos que hacer entre los tamaños de los conjuntos de datos de entrenamiento y de prueba.

Resumen

En este ejercicio hemos tratado los siguientes conceptos

Dividir un conjunto de datos en un conjunto de entrenamiento y un conjunto de prueba.<br>
Entrenamiento de un modelo utilizando el conjunto de entrenamiento y comprobación de su rendimiento en el conjunto de entrenamiento, en el conjunto de prueba y en un nuevo conjunto de datos desconocido.<br>
Comparación de los respectivos MSE para poner de relieve los efectos y peligros del sobreajuste.